In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from preprocess import OCRDataset  # Import from preprocess.py
import os
from model import OCRModel
from dataload import bangla_load, english_load



Directory: Dataset/Bangla/Dataset/Train
Checked 12000 images in total.
Removed 0 unreadable images.
Removed 0 non-image files.
Total number of images: 12000
Total number of labels: 50

Directory: Dataset/Bangla/Dataset/Test
Checked 3000 images in total.
Removed 0 unreadable images.
Removed 0 non-image files.
Total number of images: 3000
Total number of labels: 50

Directory: Dataset/English/data/training_data
Checked 20628 images in total.
Removed 0 unreadable images.
Removed 0 non-image files.
Total number of images: 20628
Total number of labels: 36

Directory: Dataset/English/data/testing_data
Checked 1008 images in total.
Removed 0 unreadable images.
Removed 0 non-image files.
Total number of images: 1008
Total number of labels: 36


In [2]:
bangla_train_loader, bangla_val_loader, bangla_test_loader, bangla_num_classes = bangla_load
english_train_loader, english_val_loader, english_test_loader, english_num_classes = english_load

In [ ]:
class OCRTrainer:
    def __init__(self, model, train_loader, val_loader, test_loader, lr=0.001):
        
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.lr = lr

        # Determine device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Assign model and move it to the device
        self.model = model.to(self.device)

        # Define loss and optimizer
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

    def train_and_validate(self, epochs=10):
        
        history = {
            "train_loss": [],
            "train_accuracy": [],
            "val_loss": [],
            "val_accuracy": []
        }

        for epoch in range(epochs):
            # Training
            self.model.train()
            running_loss = 0.0
            correct_train = 0
            total_train = 0
            for images, labels in self.train_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()

                # Training accuracy
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()

            train_loss = running_loss / len(self.train_loader)
            train_accuracy = 100 * correct_train / total_train

            # Validation
            self.model.eval()
            val_loss = 0.0
            correct_val = 0
            total_val = 0
            with torch.no_grad():
                for images, labels in self.val_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = self.model(images)
                    loss = self.criterion(outputs, labels)
                    val_loss += loss.item()

                    # Validation accuracy
                    _, predicted = torch.max(outputs, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).sum().item()

            val_loss = val_loss / len(self.val_loader)
            val_accuracy = 100 * correct_val / total_val

            # Append metrics to history
            history["train_loss"].append(train_loss)
            history["train_accuracy"].append(train_accuracy)
            history["val_loss"].append(val_loss)
            history["val_accuracy"].append(val_accuracy)

            print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
                  f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

        return history

    def test(self):
        
        self.model.eval()
        test_loss = 0.0
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                test_loss += loss.item()

                # Test accuracy
                _, predicted = torch.max(outputs, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        test_loss = test_loss / len(self.test_loader)
        test_accuracy = 100 * correct_test / total_test
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
        return test_loss, test_accuracy


In [4]:
# Common data
lr = 0.0001
epochs = 10

# Initialize the model
bangla_model = OCRModel(bangla_num_classes)

# Initialize the trainer
bangla_trainer = OCRTrainer(bangla_model, bangla_train_loader, bangla_val_loader, bangla_test_loader, lr=lr)

# Train and validate
bangla_history = bangla_trainer.train_and_validate(epochs=epochs)

# Test
test_loss, test_accuracy = bangla_trainer.test()

# Initialize the model
english_model = OCRModel(english_num_classes)

# Initialize the trainer
english_trainer = OCRTrainer(english_model, english_train_loader, english_val_loader, english_test_loader, lr=lr)


# Train and validate
english_history = english_trainer.train_and_validate(epochs=epochs)

# Test
test_loss, test_accuracy = english_trainer.test()

Epoch 1: Train Loss: 3.0611, Train Accuracy: 31.41%, Val Loss: 2.0796, Val Accuracy: 57.67%
Epoch 2: Train Loss: 1.5686, Train Accuracy: 70.19%, Val Loss: 1.1572, Val Accuracy: 79.83%
Epoch 3: Train Loss: 0.9353, Train Accuracy: 83.36%, Val Loss: 0.7822, Val Accuracy: 85.33%
Epoch 4: Train Loss: 0.6321, Train Accuracy: 88.57%, Val Loss: 0.5800, Val Accuracy: 88.00%
Epoch 5: Train Loss: 0.4519, Train Accuracy: 92.18%, Val Loss: 0.4766, Val Accuracy: 89.50%
Epoch 6: Train Loss: 0.3287, Train Accuracy: 94.59%, Val Loss: 0.3845, Val Accuracy: 92.83%
Epoch 7: Train Loss: 0.2443, Train Accuracy: 96.28%, Val Loss: 0.3355, Val Accuracy: 93.50%
Epoch 8: Train Loss: 0.1813, Train Accuracy: 97.49%, Val Loss: 0.3028, Val Accuracy: 93.17%
Epoch 9: Train Loss: 0.1377, Train Accuracy: 98.48%, Val Loss: 0.2815, Val Accuracy: 94.00%
Epoch 10: Train Loss: 0.1015, Train Accuracy: 99.00%, Val Loss: 0.2534, Val Accuracy: 94.00%
Test Loss: 0.2951, Test Accuracy: 91.90%
Epoch 1: Train Loss: 1.2169, Train Acc

In [5]:
import os
import torch
import matplotlib.pyplot as plt
import random
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import numpy as np
from sklearn.preprocessing import label_binarize


class Evaluator:
    def __init__(self, model, test_loader, num_classes, output_dir="evaluation_outputs"):
        """
        Initializes the evaluator.

        Args:
            model (torch.nn.Module): Trained model to evaluate.
            test_loader (DataLoader): DataLoader for the test dataset.
            num_classes (int): Number of classes in the dataset.
            output_dir (str): Directory to save evaluation plots and metrics.
        """
        self.model = model
        self.test_loader = test_loader
        self.num_classes = num_classes
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.output_dir = output_dir

        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

    def evaluate_and_save(self):
        """
        Evaluates the model on test data and saves visualizations and metrics to disk.
        """
        self.model.eval()
        all_images = []
        all_labels = []
        all_predictions = []
        all_probabilities = []

        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                probabilities = torch.softmax(outputs, dim=1)
                _, predictions = torch.max(outputs, 1)

                # Collect all images, labels, and predictions
                all_images.extend(images.cpu())  # Collect all test images
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predictions.cpu().numpy())
                all_probabilities.extend(probabilities.cpu().numpy())

        # Convert collected data to NumPy arrays
        all_labels = np.array(all_labels)
        all_predictions = np.array(all_predictions)
        all_probabilities = np.array(all_probabilities)

        # Save sample predictions
        self._save_sample_predictions(all_images, all_labels, all_predictions)

        # Save confusion matrix
        self._save_confusion_matrix(all_labels, all_predictions)

        # Save classification report
        self._save_classification_report(all_labels, all_predictions)

        # Save ROC curve
        self._save_roc_curve(all_labels, all_probabilities)

    def _save_sample_predictions(self, images, labels, predictions):
        """
        Randomly selects test images until there are images of 5 unique labels 
        and saves them along with their predictions and true labels to a file.
        """
        max_samples = 5  # Limit the number of unique labels
        unique_labels = {}
        selected_indices = []
        total_samples = len(labels)

        # Randomly shuffle indices
        indices = list(range(total_samples))
        random.shuffle(indices)

        # Select images with unique labels
        for idx in indices:
            label = labels[idx]
            if label not in unique_labels:
                unique_labels[label] = True
                selected_indices.append(idx)
                if len(unique_labels) == max_samples:
                    break

        # Plot and save the selected samples
        plt.figure(figsize=(15, 5))
        for i, idx in enumerate(selected_indices):
            plt.subplot(1, len(selected_indices), i + 1)
            plt.imshow(images[idx].squeeze(), cmap='gray')
            plt.title(f"True: {labels[idx]}\nPred: {predictions[idx]}")
            plt.axis('off')

        plt.tight_layout()
        file_path = os.path.join(self.output_dir, "sample_predictions.png")
        plt.savefig(file_path)
        plt.close()




    def _save_confusion_matrix(self, labels, predictions):
        """
        Saves the confusion matrix to a file.
        """
        cm = confusion_matrix(labels, predictions)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
        plt.title("Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        file_path = os.path.join(self.output_dir, "confusion_matrix.png")
        plt.savefig(file_path)
        plt.close()

    def _save_classification_report(self, labels, predictions):
        """
        Saves the classification report to a text file.
        """
        report = classification_report(labels, predictions)
        file_path = os.path.join(self.output_dir, "classification_report.txt")
        with open(file_path, "w") as f:
            f.write(report)

    def _save_roc_curve(self, labels, probabilities):
        # Binarize labels for multi-class ROC calculation
        labels_binarized = label_binarize(labels, classes=range(self.num_classes))
        fpr, tpr, _ = roc_curve(labels_binarized.ravel(), probabilities.ravel())
        roc_auc = auc(fpr, tpr)

        # Plot the ROC curve
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, label=f"Combined Classes (AUC = {roc_auc:.2f})")
        plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
        plt.title("ROC Curve (All Classes)")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend(loc="lower right")
        file_path = os.path.join(self.output_dir, "roc_curve.png")
        plt.savefig(file_path)
        plt.close()



In [6]:
import matplotlib.pyplot as plt
import os

# Function to plot and save accuracy and loss graphs
def save_training_plots(history, output_dir, model_name):
    os.makedirs(output_dir, exist_ok=True)

    # Plot Train vs Validation Accuracy
    plt.figure(figsize=(10, 6))
    plt.plot(history['train_accuracy'], label='Train Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title(f"{model_name} Model Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.legend(loc="best")
    accuracy_plot_path = os.path.join(output_dir, f"{model_name.lower()}_accuracy.png")
    plt.savefig(accuracy_plot_path)
    plt.close()

    # Plot Train vs Validation Loss
    plt.figure(figsize=(10, 6))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f"{model_name} Model Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(loc="best")
    loss_plot_path = os.path.join(output_dir, f"{model_name.lower()}_loss.png")
    plt.savefig(loss_plot_path)
    plt.close()


In [7]:
# Evaluate and save outputs
bangla_evaluator = Evaluator(bangla_model, bangla_test_loader, bangla_num_classes, output_dir="bangla_evaluation_outputs")
bangla_evaluator.evaluate_and_save()
save_training_plots(bangla_history, output_dir="bangla_evaluation_outputs", model_name="Bangla")

english_evaluator = Evaluator(english_model, english_test_loader, english_num_classes, output_dir="english_evaluation_outputs")
english_evaluator.evaluate_and_save()
save_training_plots(english_history, output_dir="english_evaluation_outputs", model_name="English")

In [8]:
import torch

# Directory to save models
model_save_dir = "saved_models"
os.makedirs(model_save_dir, exist_ok=True)

# Save Bangla model
bangla_model_path = os.path.join(model_save_dir, "bangla_model.pth")
torch.save({
    'model_state_dict': bangla_model.state_dict(),
    'optimizer_state_dict': bangla_trainer.optimizer.state_dict(),
    'history': bangla_history
}, bangla_model_path)
print(f"Bangla model saved to {bangla_model_path}")

# Save English model
english_model_path = os.path.join(model_save_dir, "english_model.pth")
torch.save({
    'model_state_dict': english_model.state_dict(),
    'optimizer_state_dict': english_trainer.optimizer.state_dict(),
    'history': english_history
}, english_model_path)
print(f"English model saved to {english_model_path}")


Bangla model saved to saved_models/bangla_model.pth
English model saved to saved_models/english_model.pth
